# Chapter 17 Future를 이용한 동시성
## 17.1 예제: 세 가지 스타일의 웹 내려받기
### 17.1.1 순차 내려받기 스크립트

In [5]:
import os
import sys
import time

import requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()
BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):
    url = f"{BASE_URL}/{cc.lower()}/{cc.lower()}.gif"
    resp = requests.get(url)
    return resp.content


def show(text):
    print(text, end=' ')
    sys.stdout.flush()


def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')

    return len(cc_list)


def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = f"\n{count} flags downloaded in {elapsed:.2f}s"
    print(msg)


main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloaded in 18.29s


### 17.1.2 `concurrent.futures`로 내려받기

In [6]:
# 예제 17-3 futures.ThreadPoolExecutor()로 스레드화된 내려받기 스크립트
from concurrent import futures

from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
    return len(list(res))

main(download_many)

FR EG BR JP CD USIR  IN TR RU BD NG ID DE MX ET PKCN  PH VN 
20 flags downloaded in 1.47s


### 17.1.3 `Future`는 어디에 있나?
암묵적으로 `Future`를 사용하지만, 이 코드에서 직접 건드리고 있지 않다.

`Future` 객체는 "대기" 중인 작업을 큐에 넣고, 완료 상태를 조사하고, 결과(혹은 예외)를 가져올 수 있도록 캡슐화한다.


`Future`객체의 실행이 완료되지 않았을 때, `concurrency.futures.Future` 객체는 결과가 나올 때까지 호출자의 스레드를 블로킹한다.


`Executor.map()`이 반환하는 반복형 객체는 `__next__()`가 실행될 때마다 각 `Future` 객체의 `result()` 메서드를 호출

`concurrent.futures.as_completed()` 함수는 `Future` 객체를 담은 반복형을 인수로 받아, 완료된 `Future` 객체를 생성한 반복자를 반환한다.

In [9]:
# 예제 17-4: download_many() 함수 안의 executor.map()을 executor.submit()과 futures.as_completed()로 대체하기
from concurrent import futures

from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    cc_list = cc_list[:5]
    # workers = min(MAX_WORKERS, len(cc_list))
    workers = 3
    with futures.ThreadPoolExecutor(workers) as executor:
        to_do = []
        # 하나씩 직접 호출해주고, Future 객체를 저장
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            print(f"Scheduler for {cc}: {future}")
        
        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            print(f"{future} result: {res}")
            results.append(res)
    
    return len(results)

main(download_many)

Scheduler for BR: <Future at 0x105525b80 state=running>
Scheduler for CN: <Future at 0x1055f4b50 state=running>
Scheduler for ID: <Future at 0x1055c6370 state=running>
Scheduler for IN: <Future at 0x1055c6250 state=pending>
Scheduler for US: <Future at 0x1055f4850 state=pending>
ID <Future at 0x1055c6370 state=finished returned str> result: ID
BR CN <Future at 0x105525b80 state=finished returned str> result: BR
<Future at 0x1055f4b50 state=finished returned str> result: CN
IN <Future at 0x1055c6250 state=finished returned str> result: IN
US <Future at 0x1055f4850 state=finished returned str> result: US

5 flags downloaded in 1.05s


스레드가 3개이기 때문에 첫 세 개만 `state=running`이다. 나머지 2개 `Future` 객체는 대기 중이다

## 17.2 블로킹 I/O와 GIL

## 17.3 `concurrent.futures`로 프로세스 실행하기
- `ProcessPoolExecutor()` 클래스로 다중 프로세스도 가능하다.
- 인자를 넘기지 않으면 `os.cpu_count()`가 반환하는 값을 사용한다.

## 17.4 `Excutor.map()` 실험

In [4]:
# 예제 17-6 ThreadPoolExecutor() 메서드 사용 예
from concurrent import futures
from time import sleep, strftime


def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)


def loiter(n):
    msg = "{}loiter({}): doing nothing for {}s..."
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = "{}loiter({}): done."
    display(msg.format('\t'*n, n))
    return n * 10


def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))
        
# main() -> 주피터에서 실행하면 출력이 겹쳐서 나옴

~~~
[20:56:19] Script starting.
[20:56:19] loiter(0): doing nothing for 0s...
[20:56:19] loiter(0): done.
[20:56:19]      loiter(1): doing nothing for 1s...
[20:56:19]              loiter(2): doing nothing for 2s...
[20:56:19]                      loiter(3): doing nothing for 3s...
[20:56:19] results: <generator object Executor.map.<locals>.result_iterator at 0x10525b660>
[20:56:19] Waiting for individual results:
[20:56:19] result 0: 0
[20:56:20]      loiter(1): done.
[20:56:20]                              loiter(4): doing nothing for 4s...
[20:56:20] result 1: 10
[20:56:21]              loiter(2): done.
[20:56:21] result 2: 20
[20:56:22]                      loiter(3): done.
[20:56:22] result 3: 30
[20:56:24]                              loiter(4): done.
[20:56:24] result 4: 40
~~~

- `Executor.map()`은 함수를 호출한 순서대로 결과를 반환한다. 따라서 첫 번째로 호출한 함수가 실행 시간이 길다면, 결과를 오래 기다려야 할 수 있다.
- `submit()` 메서드와 `futures.as_completed()` 함수를 함께 사용해야 한다.

## 17.5 진행 상황 출력하고 에러를 처리하며 내려받기